In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [3]:
# !pip install dill
import dill

In [4]:
# to store the session
dill.dump_session('notebook_env.db')
# dill.load_session('notebook_env.db')

In [5]:
import tensorflow as tf

In [6]:
filepath="/kaggle/input/digit-recognizer/"

train_data = pd.read_csv(os.path.join(filepath,"train.csv"))

In [7]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split
all_train, all_valid = train_test_split(train_data, test_size=0.2, random_state=42)

In [9]:
y_train = all_train["label"]
x_train = all_train.drop(["label"], axis=1)

In [10]:
y_valid = all_valid["label"]
x_valid = all_valid.drop(["label"], axis=1)

In [11]:
x_train, x_valid = x_train / 255.0, x_valid / 255.0

In [12]:
x_train

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
34941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
x_train_num = x_train.to_numpy()

In [20]:
x_train_num.shape

(33600, 784)

In [22]:
x_train_res = x_train_num.reshape([33600,28,28])

In [32]:
x_valid_num = x_valid.to_numpy()

In [35]:
x_valid_num.shape

(8400, 784)

In [36]:
x_valid_res = x_valid_num.reshape([8400,28,28])

## TensorFlow & Keras

In [24]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [27]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
loss_fn(y_train[:1], predictions).numpy()

2.4963698

In [29]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [31]:
model.fit(x_train_res, y_train, epochs=5)

Train on 33600 samples
Epoch 1/5
33600/33600 [==============================] - 5s 145us/sample - loss: 0.3695 - accuracy: 0.8936
Epoch 2/5
33600/33600 [==============================] - 4s 130us/sample - loss: 0.1783 - accuracy: 0.9483
Epoch 3/5
33600/33600 [==============================] - 4s 128us/sample - loss: 0.1315 - accuracy: 0.9609
Epoch 4/5
33600/33600 [==============================] - 4s 125us/sample - loss: 0.1048 - accuracy: 0.9684
Epoch 5/5
33600/33600 [==============================] - 4s 126us/sample - loss: 0.0879 - accuracy: 0.9728


In [37]:
model.evaluate(x_valid_res,  y_valid, verbose=2)

8400/8400 - 1s - loss: 0.0996 - accuracy: 0.9711


[0.09955079235313903, 0.9710714]

## Make submission.csv

In [38]:
filepath="/kaggle/input/digit-recognizer/"

test_data = pd.read_csv(os.path.join(filepath,"test.csv"))

In [40]:
test_data_num = test_data.to_numpy()

In [42]:
test_data_num.shape

(28000, 784)

In [43]:
test_data_res = test_data_num.reshape([28000,28,28])

In [55]:
predictions = model(test_data_res).numpy()

In [56]:
predictions

array([[ -813.3907  , -2401.2244  ,  2884.8052  , ...,  -379.45758 ,
         -681.31    , -2109.6523  ],
       [ 3171.8015  , -1595.476   ,   514.7331  , ...,  -342.03488 ,
        -1689.6274  , -1019.6063  ],
       [-1362.183   ,  -456.67184 ,  -697.6052  , ...,  -376.47656 ,
          145.26433 ,  1273.4647  ],
       ...,
       [-2218.9294  ,  -699.9026  ,  -776.12286 , ..., -1654.807   ,
          314.3633  ,   846.022   ],
       [-1539.122   , -2806.522   , -1664.867   , ...,   -10.904142,
         -676.20135 ,  2355.7217  ],
       [ -798.66364 , -2147.533   ,  2508.471   , ..., -1410.515   ,
         -514.9973  , -1403.7351  ]], dtype=float32)

In [60]:
test_labels = np.argmax(predictions, axis=1)

In [46]:
filepath="/kaggle/input/digit-recognizer/"

sample_data = pd.read_csv(os.path.join(filepath,"sample_submission.csv"))

In [49]:
sample_data

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [61]:
output = pd.DataFrame({'ImageId': sample_data.ImageId, 'Label': test_labels})
output.to_csv('my_submission.csv', index=False)